In [204]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Bài toán hồi quy tuyến tính (Liner Regression)
Sếp đưa cho bạn một bảng dữ liệu số có các trường $x_1, x_2, ..., x_m, y$  
Dữ liệu dòng thứ $i$ là $x_1^{(i)}, x_2^{(i)}, ... , x_m^{(i)}, y^{(i)}$
Nhiệm vụ của bạn, căn cứ vào dữ liệu đã cho, bạn nhập một input $x_1^{(k)}, x_2^{(k)}, ... , x_m^{(k)}$ và mô hình của bạn phải trả ra được $y^{(k)}$ gần đúng  
Để giải quyết, bạn có ý tưởng là xây dựng một hàm số $y = w_0 + w_1.x_1 + ... + w_m.x_m (*)$  
Và bây giờ, nhiệm vụ của bạn là đi tìm $w_0, w_1, ..., w_m$

Nếu để ý thì phương trình $(*)$ là phương trình tuyến tính, nên bài toán người được gọi là bài toán hồi quy tuyến tính  


### Giá trị dự đoán của model
$\hat{y}_1 = w_0 + w_1.x_1^{(1)} + ... + w_m.w_m^{(1)}$  
...  
$\hat{y}_i = w_0 + w_1.x_1^{(i)} + ... + w_m.w_m^{(i)}$  
...  
$\hat{y}_N = w_0 + w_1.x_1^{(N)} + ... + w_m.w_m^{(N)}$  


Ta có thể sử dụng ma trận để viết lại công thức gọn hơn.  

Đặt:
$X = \begin{bmatrix} 1 & x_1^{(1)} & ... & x_m^{(1)} \\ 1 & x_1^{(2)} & ... & x_m^{(2)} \\ ... & ... & ... & ... \\ 1 & x_1^{(N)} & ... & x_m^{(N)} \end{bmatrix}_{N \times M}$,
$W = \begin{bmatrix} w_0 \\ w_1 \\ ... \\ w_m \end{bmatrix}_{M \times 1}$,
$\hat{Y} = \begin{bmatrix} \hat{y_1} \\ \hat{y_2} \\ ... \\ \hat{y_N} \end{bmatrix}_{N \times 1}$  

Khi đó: $\hat{Y} = X \times W$  

Nhiệm vụ trở thành tìm $W$ để $\hat{Y}$ giống $Y$ nhất



In [205]:
def get_value(w, X):
    X = np.hstack((np.ones((X.shape[0], 1)), X))
    return X @ w

### Hàm mất mát
Giá trị sai số trên mỗi mẫu dữ liệu: $L_i = (\hat{y_i} - y_i)^2 $  
Hàm mất mát (loss function): $J(x) = \sum_{i = 1}^N(L_i) = \sum_{i = 1}^N(\hat{y_i} - y_i)^2$  
Mục tiêu là đi tối thiểu hóa hàm mất mát với bộ dữ liệu đã cho

In [206]:
def get_loss_function(w, data):
    X = data[:, :-1]
    y = data[:, -1].reshape(-1, 1)
    Y = get_value(w, X)
    return np.sum((Y - y) ** 2)

### Vector Gradient
Đạo hàm của hàm số $f(x)$ cho ta biết độ dốc của đồ thị và hướng tăng của hàm số.  
Ví dụ: $f(x) = x^2 $ có $f'(x) = 2x$  
Xét dấu của $f'(2) = 4 > 0$ như vậy khi tăng giá trị $x$ thì $f(x)$ sẽ tăng.  
Xét dấu của $f'(-2) = -4 < 0$ như vậy khi giảm giá trị $x$ thì $f(x)$ sẽ tăng.  

Tổng quát hơn, xét hàm nhiều biến $f(x_1, ..., x_n)$ với $x_i \in R$  
Đạo hàm riêng theo $x_i$ tại $M(a_1, ..., a_n) \in R^n$ là $\frac{\partial f}{\partial x_i}(M)$.  
Dấu của $\frac{\partial f}{\partial x_i}(M)$ cho biết hướng $a_i$ thay đổi để giá trị $f(a_0, a_1, ..., a_n)$ tăng  

Vector Gradient tại $M$: $\vec{\triangledown}(M) = \begin{bmatrix} \frac{\partial f}{\partial x_1}(M) & ... & \frac{\partial f}{\partial x_n}(M) \end{bmatrix}^T $ cho biết hướng $M$ di chuyển để $f(M)$ tăng.

### Xác định Vector Gradient cho hàm loss funtion $J(x)$
Với $J(x) = \sum_{i = 1}^N(L_i) = \sum_{i = 1}^N(\hat{y_i} - y_i)^2$  
Giả sử với lời giải hiện tại là $M^{(k)}(w_0, w_1, ..., w_m)$  

$\vec{\triangledown}(M^{(k)}) = \begin{bmatrix} \frac{\partial J}{\partial w_0}(M^{(k)}) & ... & \frac{\partial J}{\partial x_m}(M^{(k)}) \end{bmatrix}^T $

Để cho gọn thì chúng ta bỏ $M^{(i)}$, và những đạo hàm sau, ta ngầm hiểu là tính tại $M^{(i)}$:  
$\vec{\triangledown} = \begin{bmatrix} \frac{\partial J}{\partial w_0} & ... & \frac{\partial J}{\partial x_m} \end{bmatrix}^T $
+ $\frac{\partial J}{\partial w_j} = \sum_{i = 1}^N\frac{\partial L_i}{\partial w_j}$  
+ $\frac{\partial L_i}{\partial w_j} = \frac{\partial L_i}{\partial \hat{y}^{(i)}} \times \frac{\partial \hat{y}^{(i)}}{\partial w_j} $
    + $\frac{\partial L_i}{\partial \hat{y}^{(i)}} = 2(\hat{y}^{(i)} - y^{(i)})$
    + $\frac{\partial \hat{y}^{(i)}}{\partial w_j} = x_j^{(i)}$

Thế ngược trở lại ta được: $\frac{\partial J}{\partial w_j} = 2\sum_{i = 1}^N(x_j^{(i)}.(\hat{y}^{(i)} - y^{(i)}))$  

Với phép đặt ban đầu: $\vec{\triangledown} = 2 X^T . (\hat{Y} - Y)$ 


In [207]:
def gradient(point, data):
    X = data[:, :-1]
    y = data[:, -1].reshape(-1, 1)
    Y = get_value(point, X)
    X = np.hstack((np.ones((X.shape[0], 1)), X))
    return 2 * X.T @ (Y - y)

In [208]:
class Model:
    def __init__(self):
        self.w = None
        self.loss_function = None

    def gen_model(self, data):
        self.w = np.random.uniform(0, 1, data.shape[1]).reshape(-1, 1)

    def cal_loss_function(self, data):
        self.loss_function = get_loss_function(self.w, data)
    
    def cal_value(self, inp):
        return get_value(self.w, inp)
    
    def show(self):
        print(f"w: {self.w.reshape(1, -1)}")
        print(f"loss_function: {self.loss_function}")

Giả sử hiện tại ta đang có lời giải $M^{(k)}$, để tối thiểu hóa hàm mất mát $J(x)$, ta đi ngược lại với $\vec{\triangledown}(M^{(k)})$.  
$M^{(k + 1)} = M^{(k)} - \alpha . \vec{\triangledown}(M^{(k)}) $ với $\alpha$ là hệ số học (learn_rate)

In [209]:
def train(model, data, learn_rate, max_inter):
    model.gen_model(data)
    history = []
    for i in range(max_inter):
        model.w -= learn_rate * gradient(model.w, data)
        model.cal_loss_function(data)
        history.append(model.loss_function)
        if model.loss_function < 1e-24:
            break
    return model, history


In [ ]:
# Data cần được định dạng với cột cuối là kết quả
data = np.array([[1, 2], [3, 3], [5, 4], [7, 5], [9, 6]])

model = Model()
learn_rate = 0.0005
max_inter = 100000

model, history = train(model, data, learn_rate, max_inter)

for i in range(len(history)):
    print(f"Generation {i}, loss_function = {history[i]}")

print("w: ", model.w)
print("Loss function: ", model.loss_function)


Generation 0, loss_function = 41.36164702931079
Generation 1, loss_function = 28.69531424210024
Generation 2, loss_function = 19.944265738014444
Generation 3, loss_function = 13.89816272511359
Generation 4, loss_function = 9.720820800440492
Generation 5, loss_function = 6.834546874261772
Generation 6, loss_function = 4.840231246428968
Generation 7, loss_function = 3.4621417749722534
Generation 8, loss_function = 2.5097839309926693
Generation 9, loss_function = 1.851551129147774
Generation 10, loss_function = 1.39652058284872
Generation 11, loss_function = 1.0818766422332393
Generation 12, loss_function = 0.864221980192011
Generation 13, loss_function = 0.7135747683994456
Generation 14, loss_function = 0.6092215180218838
Generation 15, loss_function = 0.5368519398634742
Generation 16, loss_function = 0.48657951078135664
Generation 17, loss_function = 0.45157394626426034
Generation 18, loss_function = 0.427116419429168
Generation 19, loss_function = 0.40994684201158327
Generation 20, los